In [1]:
import pandas as pd
voting_df = pd.read_csv(filepath_or_buffer='/kaggle/input/2020-us-presidential-election-results-by-state/voting.csv')
voting_df.head(n=5)

,state,state_abr,trump_pct,biden_pct,trump_vote,biden_vote,trump_win,biden_win
0,Alaska,AK,53.1,43.0,189543,153502,1,0
1,Hawaii,HI,34.3,63.7,196864,366130,0,1
2,Washington,WA,39.0,58.4,1584651,2369612,0,1
3,Oregon,OR,40.7,56.9,958448,1340383,0,1
4,California,CA,34.3,63.5,5982194,11082293,0,1


In [2]:
college_df = pd.read_csv(filepath_or_buffer='/kaggle/input/electoralcollege/Electoral_College.csv', usecols=['Year', 'State', 'Votes'])
college_df['State'] = college_df['State'].replace(to_replace={'D.C.': 'District of Columbia'})
college_df.head(n=5)

,Year,State,Votes
0,1788,Alabama,NaN
1,1792,Alabama,NaN
2,1796,Alabama,NaN
3,1800,Alabama,NaN
4,1804,Alabama,NaN


In [3]:
ec2020_df = college_df[college_df['Year']==2020].drop(columns=['Year',]).copy()
ec2020_df['Votes'] = ec2020_df['Votes'].astype(int)
ec2020_df.head(n=5)

,State,Votes
58,Alabama,9
117,Alaska,3
176,Arizona,11
235,Arkansas,6
294,California,55


In [4]:
result_2020_df = voting_df[['state', 'trump_win', 'biden_win']].rename(columns={'state' : 'State'}).merge(how='inner', on='State', right=ec2020_df).copy()
result_2020_df['DEC votes'] = result_2020_df['biden_win'] * result_2020_df['Votes']
result_2020_df['REC votes'] = result_2020_df['trump_win'] * result_2020_df['Votes']
result_2020_df.head(n=5)

,State,trump_win,biden_win,Votes,DEC votes,REC votes
0,Alaska,1,0,3,0,3
1,Hawaii,0,1,4,4,0
2,Washington,0,1,12,12,0
3,Oregon,0,1,7,7,0
4,California,0,1,55,55,0


Now we can check the result against the historical result.

In [5]:
result_2020_df['DEC votes'].sum(), result_2020_df['REC votes'].sum()

(306, 232)

We can introduce the changes to the Electoral College due to the 2020 Census.

In [6]:
changes = {
    'California': -1,
    'Colorado': 1,
    'Florida' : 1,
    'Illinois': -1,
    'Michigan': -1,
    'Montana': 1,
    'New York': -1,
    'North Carolina': 1,
    'Ohio': -1,
    'Oregon': 1,
    'Pennsylvania': -1,
    'Texas': 2,
    'West Virginia': -1
}
# there should be no net changes
sum(list(changes.values()))

0

In [7]:
result_2024_df = result_2020_df.copy().drop(columns=['DEC votes', 'REC votes'])
result_2024_df['Votes'] = result_2020_df.apply(axis=1, func=lambda x: x['Votes'] if x['State'] not in changes.keys() else x['Votes'] + changes[x['State']])
result_2024_df.head()

,State,trump_win,biden_win,Votes
0,Alaska,1,0,3
1,Hawaii,0,1,4
2,Washington,0,1,12
3,Oregon,0,1,8
4,California,0,1,54


Our base scenario is that no states flip and the only change is due to reapportionment.

In [8]:
result_2024_df['DEC votes'] = result_2024_df['biden_win'] * result_2024_df['Votes']
result_2024_df['REC votes'] = result_2024_df['trump_win'] * result_2024_df['Votes']

result_2024_df['DEC votes'].sum(), result_2024_df['REC votes'].sum()

(303, 235)

D 303 - 235 R is our base assumption.